# Cleaning Employee Data with Custom Parsing

In [18]:
import pandas as pd
import numpy as np
import re

## Step 1: Load raw data

In [19]:
df = pd.read_csv("employee_data.csv")

## Step 2: Replace all corrupted characters

In [20]:
df.replace(to_replace=r'\+AC0-', value='-', regex=True, inplace=True)
df.replace(to_replace=r'\+ACO-', value='-', regex=True, inplace=True)

## Step 3: Define a custom parser for corrupted dates

In [21]:
def parse_date(date_str):
    try:
        if isinstance(date_str, str):
            date_str = date_str.replace("+AC0-", "-").replace("+ACO-", "-")
            date_str = re.sub(r"[^\w\-]", "", date_str)  # removing the remaining junk
            for fmt in ("%Y-%m-%d", "%d-%b-%y", "%d-%m-%Y"):
                try:
                    return pd.to_datetime(date_str, format=fmt)
                except:
                    continue
        return pd.NaT
    except:
        return pd.NaT

## Step 4: Apply parser to date columns

In [22]:
for col in ['StartDate', 'ExitDate', 'DOB']:
    df[col] = df[col].apply(parse_date)

## Step 5: Convert numeric columns safely

In [23]:
for col in ['Age', 'Experience', 'Salary']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

## Step 6: Drop rows missing essential fields

In [27]:
df.dropna(subset=['StartDate', 'Age', 'Salary'], inplace=True)
df["ExitDate"] = df["ExitDate"].fillna("Still Working")

## Step 7: Review cleaned data

In [28]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   EmpID                       3000 non-null   int64         
 1   FirstName                   3000 non-null   object        
 2   LastName                    3000 non-null   object        
 3   StartDate                   3000 non-null   datetime64[ns]
 4   ExitDate                    3000 non-null   object        
 5   EmployeeStatus              3000 non-null   object        
 6   EmployeeType                3000 non-null   object        
 7   EmployeeClassificationType  3000 non-null   object        
 8   DepartmentType              3000 non-null   object        
 9   DOB                         3000 non-null   datetime64[ns]
 10  JobFunctionDescription      3000 non-null   object        
 11  GenderCode                  3000 non-null   object      

## Step 8: Save cleaned data

In [29]:
df.to_csv('cleaned_hr_data.csv', index=False)